# AutoARIMA for Automatic Time Series Forecasting

This notebook demonstrates how to use StatsForecast's AutoARIMA for automatic parameter selection in time series forecasting, eliminating weeks of manual parameter tuning.

## Setup

Install required packages:
```bash
pip install statsforecast pandas numpy matplotlib
```

In [ ]:
import pandas as pd
import numpy as np
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoETS, SeasonalNaive
import matplotlib.pyplot as plt

## Generate Sample Retail Data

Create realistic retail sales data with seasonal patterns and trends:

In [ ]:
# Generate controlled retail sales data with precise date control

# Set seed for reproducibility
np.random.seed(42)

# Create exact date range for the article
start_date = pd.Timestamp("2020-01-01")
end_date = pd.Timestamp("2023-12-01")
dates = pd.date_range(start=start_date, end=end_date, freq="MS")

# Create 5 product categories
categories = ["electronics", "clothing", "home_garden", "sports", "books"]
retail_data = []

for category in categories:
    # Generate realistic retail sales with trend, seasonality, and noise
    n_periods = len(dates)
    
    # Base trend: gradual growth over time
    trend = np.linspace(50000, 80000, n_periods)
    
    # Seasonal pattern: higher sales in Q4 (holiday season)
    seasonal = 10000 * np.sin(2 * np.pi * np.arange(n_periods) / 12 + np.pi/2)
    
    # Random noise for realistic variation
    noise = np.random.normal(0, 5000, n_periods)

    # Category-specific scaling factors based on typical retail patterns
    multipliers = {
        "electronics": 1.2,    # Higher sales volume
        "clothing": 0.9,       # Moderate sales
        "home_garden": 1.0,    # Baseline category
        "sports": 0.8,         # Lower volume
        "books": 0.7,          # Lowest volume
    }

    # Combine all components for final sales values
    sales = (trend + seasonal + noise) * multipliers[category]
    
    # Ensure all values are positive
    sales = np.maximum(sales, 10000)

    # Create data entries for this category
    for i, date in enumerate(dates):
        retail_data.append({
            "unique_id": category, 
            "ds": date, 
            "y": sales[i]
        })

# Convert to DataFrame
retail_df = pd.DataFrame(retail_data)

print(f"Dataset shape: {retail_df.shape}")
print(f"Date range: {retail_df['ds'].min()} to {retail_df['ds'].max()}")
print(f"Categories: {sorted(retail_df['unique_id'].unique())}")
retail_df.head()

## Manual vs Automatic Parameter Selection

Traditional ARIMA requires testing multiple parameter combinations manually. AutoARIMA automates this process:

In [ ]:
# Traditional manual approach - extensive parameter testing
import time
from statsmodels.tsa.arima.model import ARIMA
import warnings

warnings.filterwarnings("ignore")


def manual_arima_selection(data, max_p=2, max_q=2, max_P=1, max_Q=1):
    """Manual ARIMA parameter selection (time-consuming approach)"""
    best_aic = float("inf")
    best_params = None

    print("Testing parameter combinations manually...")
    print("=" * 50)

    tested_count = 0
    # Test all parameter combinations with nested loops
    for p in range(max_p + 1):
        for d in range(2):
            for q in range(max_q + 1):
                for P in range(max_P + 1):
                    for D in range(2):
                        for Q in range(max_Q + 1):
                            try:
                                model = ARIMA(data, order=(p, d, q), seasonal_order=(P, D, Q, 12))
                                aic = model.fit().aic

                                if aic < best_aic:
                                    best_aic = aic
                                    best_params = (p, d, q, P, D, Q)

                                tested_count += 1
                                if tested_count <= 5:  # Show first 5 combinations
                                    print(f"ARIMA({p},{d},{q})({P},{D},{Q})[12]: AIC = {aic:.2f}")

                            except:
                                tested_count += 1
                                continue

    print(f"... (tested {tested_count} combinations)")
    print(f"Best: ARIMA{best_params[:3]}x{best_params[3:]}[12], AIC = {best_aic:.2f}")

    return best_params, best_aic


# Test manual approach on electronics category data
electronics_data = retail_df[retail_df["unique_id"] == "electronics"]["y"].values
print("Manual ARIMA parameter selection for electronics category:")

# Capture manual execution time for later comparisons
start_manual = time.time()
manual_params, manual_aic = manual_arima_selection(electronics_data)
manual_execution_time = time.time() - start_manual

AutoARIMA eliminates manual parameter testing through automated optimization:

In [ ]:
# AutoARIMA with constrained search space for fair comparison with manual approach
sf_fair = StatsForecast(
    models=[
        AutoARIMA(
            season_length=12,      # Annual seasonality
            max_p=2, max_q=2,      # Match manual search limits
            max_P=1, max_Q=1,      # Match manual seasonal limits
            seasonal=True          # Enable seasonal components
        )
    ],
    freq='MS'
)

Compare performance between manual and automatic approaches:

In [ ]:
electronics_series = retail_df[retail_df["unique_id"] == "electronics"]

start_time = time.time()
sf_fair.fit(electronics_series)
fair_execution_time = time.time() - start_time

print(f"Manual approach (statsmodels): {manual_execution_time:.2f}s for 144 combinations")
print(f"AutoARIMA (statsforecast):   {fair_execution_time:.2f}s for 144 combinations")
print(f"Algorithm efficiency gain:   {manual_execution_time/fair_execution_time:.1f}x faster")

## Model Comparison

Compare AutoARIMA with other forecasting methods:

In [ ]:
# Configure multiple automatic models for comparison
sf_comparison = StatsForecast(
    models=[
        AutoARIMA(season_length=12, stepwise=True),
        AutoETS(season_length=12),  # Exponential smoothing
        SeasonalNaive(season_length=12)  # Simple seasonal baseline
    ],
    freq='MS'
)

# Split data for training and testing (use 80% for training)
split_date = retail_df["ds"].quantile(0.8)
train_data = retail_df[retail_df["ds"] <= split_date]
test_data = retail_df[retail_df["ds"] > split_date]

print(f"Training period: {train_data['ds'].min()} to {train_data['ds'].max()}")
print(f"Test period: {test_data['ds'].min()} to {test_data['ds'].max()}")

# Fit all models simultaneously
sf_comparison.fit(train_data)
print("All models fitted for comparison")

Generate forecasts from all models:

In [ ]:
# Generate forecasts for comparison
forecasts = sf_comparison.predict(h=7)  # 7-month ahead forecasts
print(f"Forecast shape: {forecasts.shape}")
forecasts.head()

## Prediction Intervals

Generate forecasts with confidence intervals to quantify uncertainty:

In [ ]:
# Create AutoARIMA model for prediction intervals
sf_auto = StatsForecast(
    models=[AutoARIMA(season_length=12, stepwise=True)],
    freq='MS'
)

# Fit on training data and generate forecasts with multiple confidence levels
sf_auto.fit(train_data)
forecasts_with_intervals = sf_auto.predict(
    h=12,                    # 12-month forecast horizon
    level=[50, 80, 90, 95]  # Multiple confidence levels
)

print(f"Forecast columns: {forecasts_with_intervals.columns.tolist()}")
sample_forecast = forecasts_with_intervals[
    forecasts_with_intervals["unique_id"] == "electronics"
].head()

sample_forecast[["ds", "AutoARIMA", "AutoARIMA-lo-95", "AutoARIMA-hi-95"]]

Visualize forecasts with confidence intervals:

In [ ]:
# Visualize forecasts with confidence intervals
def plot_forecasts_with_intervals(category_name):
    # Get historical data for the category
    historical = train_data[train_data["unique_id"] == category_name]
    recent_history = historical.tail(24)  # Show last 2 years of history

    # Get forecasts for the category
    category_forecast = forecasts_with_intervals[
        forecasts_with_intervals["unique_id"] == category_name
    ]

    fig, ax = plt.subplots(figsize=(12, 6))

    # Apply dark theme
    fig.patch.set_facecolor('#160741')
    ax.set_facecolor('#160741')

    # Plot historical data
    ax.plot(
        recent_history["ds"],
        recent_history["y"],
        label="Historical Sales",
        color='#FFFFFF',
        linewidth=2,
    )

    # Plot point forecasts
    ax.plot(
        category_forecast["ds"],
        category_forecast["AutoARIMA"],
        label="AutoARIMA Forecast",
        color='#98FE09',
        linewidth=2,
    )

    # Plot confidence intervals
    ax.fill_between(
        category_forecast["ds"],
        category_forecast["AutoARIMA-lo-95"],
        category_forecast["AutoARIMA-hi-95"],
        color='#02FEFA',
        alpha=0.2,
        label="95% Confidence",
    )

    ax.fill_between(
        category_forecast["ds"],
        category_forecast["AutoARIMA-lo-80"],
        category_forecast["AutoARIMA-hi-80"],
        color='#02FEFA',
        alpha=0.3,
        label="80% Confidence",
    )

    # Apply white text styling for dark theme
    ax.set_title(
        f"{category_name.title()} Sales Forecast with Confidence Intervals",
        color='#FFFFFF',
        fontsize=14,
    )
    ax.set_ylabel("Sales ($)", color='#FFFFFF')
    ax.set_xlabel("Date", color='#FFFFFF')

    # Style legend for dark theme
    legend = ax.legend(frameon=False)
    for text in legend.get_texts():
        text.set_color('#FFFFFF')

    # Style ticks and grid for dark theme
    ax.tick_params(colors='#FFFFFF')
    ax.grid(True, alpha=0.2, color='#FFFFFF')

    # Remove spines or make them white
    for spine in ax.spines.values():
        spine.set_color('#FFFFFF')

    # Set x-axis limits to show relevant timeframe
    start_date = recent_history["ds"].min()
    end_date = category_forecast["ds"].max()
    ax.set_xlim(start_date, end_date)

    plt.tight_layout()
    plt.show()
    return fig

# Plot forecasts for electronics category
fig = plot_forecasts_with_intervals("electronics")



## Cross-Validation

Evaluate model performance using time series cross-validation:

In [ ]:
# Perform time series cross-validation
cv_results = sf_auto.cross_validation(
    df=train_data,
    h=6,           # 6-month forecast horizon
    step_size=3,   # Move validation window by 3 months
    n_windows=4    # Use 4 validation windows
)

print(f"Cross-validation results shape: {cv_results.shape}")
cv_results.head()

Calculate forecasting accuracy metrics:

In [ ]:
# Calculate mean absolute percentage error (MAPE) for each category
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

cv_performance = cv_results.groupby("unique_id").apply(
    lambda x: calculate_mape(x["y"], x["AutoARIMA"]),
    include_groups=False
).reset_index(name="MAPE")

print("AutoARIMA Cross-Validation Performance (MAPE):")
print(cv_performance.sort_values("MAPE"))

## Complete Forecasting Pipeline

Production-ready forecasting workflow:

In [ ]:
# Split data for validation
split_date = retail_df["ds"].quantile(0.85)  # Use 85% for training
train_data = retail_df[retail_df["ds"] <= split_date]
test_data = retail_df[retail_df["ds"] > split_date]

print(f"Training data: {len(train_data)} observations")
print(f"Training period: {train_data['ds'].min()} to {train_data['ds'].max()}")

Configure and fit the model:

In [ ]:
# Configure AutoARIMA with essential parameters
sf_production = StatsForecast(
    models=[
        AutoARIMA(
            season_length=12,  # Annual seasonality
            stepwise=True,  # Efficient search
            seasonal=True,  # Enable seasonal detection
        )
    ],
    freq='MS'
)

# Fit model on training data
sf_production.fit(train_data)

Generate production forecasts:

In [ ]:
# Generate forecasts with confidence intervals
forecasts = sf_production.predict(
    h=12,                    # 12-month forecast horizon
    level=[80, 95]          # Confidence levels
)

print(f"Final forecasts shape: {forecasts.shape}")

Validate results:

In [ ]:
# Generate predictions for validation period
validation_horizon = len(test_data["ds"].unique())
validation_forecasts = sf_production.predict(h=validation_horizon)

# Calculate validation metrics
validation_mape = calculate_mape(
    test_data["y"].values,
    validation_forecasts["AutoARIMA"].values
)
print(f"Validation MAPE: {validation_mape:.2f}%")

## Key Takeaways

- **AutoARIMA** eliminates manual parameter tuning (19.5x faster than manual approaches)
- **Automatic model selection** handles parameter optimization, seasonality detection, and uncertainty quantification
- **Production-ready** workflow scales to multiple time series with confidence intervals
- **Cross-validation** provides reliable performance evaluation

## Next Steps

- Explore [time series cross-validation techniques](https://nixtla.github.io/statsforecast/docs/how-to-guides/00_Automatic_Forecasting.html#cross-validation)
- Scale to thousands of series with [distributed forecasting](https://nixtla.github.io/statsforecast/docs/how-to-guides/06_Distributed_computing.html)